## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Half Moon Bay,US,37.4636,-122.4286,78.75,78,1,1.01,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,38.86,75,75,2.30,broken clouds
2,2,Caravelas,BR,-17.7125,-39.2481,68.31,68,2,5.44,clear sky
3,3,Adrar,MR,20.5022,-10.0711,89.67,22,73,9.24,broken clouds
4,4,Vostok,RU,46.4856,135.8833,93.45,36,49,4.45,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation "))
max_temp = float(input("What is the maximum temperature you would like for your vacation "))


What is the minimum temperature you would like for your vacation 60
What is the maximum temperature you would like for your vacation 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
filtered_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Caravelas,BR,-17.7125,-39.2481,68.31,68,2,5.44,clear sky
5,5,Srednekolymsk,RU,67.4500,153.6833,64.04,40,100,13.53,overcast clouds
12,12,Vao,NC,-22.6667,167.4833,72.36,80,10,7.40,clear sky
15,15,Kutum,SD,14.2000,24.6667,72.07,77,98,10.85,overcast clouds
17,17,Taunggyi,MM,20.7833,97.0333,69.03,79,100,5.03,overcast clouds
23,23,College,US,64.8569,-147.8028,70.00,53,75,10.36,broken clouds
26,26,Albany,US,42.6001,-73.9662,72.18,70,100,3.00,overcast clouds
27,27,Yar-Sale,RU,66.8333,70.8333,60.21,68,9,13.58,clear sky
28,28,Ambilobe,MG,-13.2000,49.0500,64.92,74,78,2.82,broken clouds
33,33,Busselton,AU,-33.6500,115.3333,64.04,67,71,19.28,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.count()

City_ID                164
City                   164
Country                162
Lat                    164
Lng                    164
Max Temp               164
Humidity               164
Cloudiness             164
Wind Speed             164
Current Description    164
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_cities_df = filtered_cities_df.dropna()
filtered_cities_df.count()

City_ID                162
City                   162
Country                162
Lat                    162
Lng                    162
Max Temp               162
Humidity               162
Cloudiness             162
Wind Speed             162
Current Description    162
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Caravelas,BR,68.31,clear sky,-17.7125,-39.2481,
5,Srednekolymsk,RU,64.04,overcast clouds,67.4500,153.6833,
12,Vao,NC,72.36,clear sky,-22.6667,167.4833,
15,Kutum,SD,72.07,overcast clouds,14.2000,24.6667,
17,Taunggyi,MM,69.03,overcast clouds,20.7833,97.0333,
23,College,US,70.00,broken clouds,64.8569,-147.8028,
26,Albany,US,72.18,overcast clouds,42.6001,-73.9662,
27,Yar-Sale,RU,60.21,clear sky,66.8333,70.8333,
28,Ambilobe,MG,64.92,broken clouds,-13.2000,49.0500,
33,Busselton,AU,64.04,broken clouds,-33.6500,115.3333,


In [ ]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file, index_label= "City_ID")

In [ ]:
vacation_df = pd.read_csv("./weather_data/WeatherPy_Vacation.csv")
vacation_df.head()

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd> at{Max Temp}</dd> 
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer) 
fig
# 11b. Display the figure
